In [1]:
import os
import numpy as np
import sys
import cv2
import argparse
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
from collections import OrderedDict
from backbones.linear import ArcfaceLinear_mbf

In [2]:
def read_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (112, 112))

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ori_img = img.copy()
    img = np.transpose(img, (2, 0, 1))
    img = torch.from_numpy(img).unsqueeze(0).float()
    img.div_(255).sub_(0.5).div_(0.5)
    return img,ori_img

In [3]:
img_path = "FairFace/aligned_fair/val/1.jpg"

In [11]:
label_dict={0:'White',1:'Black',2:'Asian',3:'Indian',4:'Others',5:'Others'}

In [14]:
#torch
net='mbf'
model_path = "race-mbf-arcface-sgd_221209_nf.pth"
backbone = ArcfaceLinear_mbf(pretrained_path='', net =net, num_class=6, num_features=512,freeze=False,fp16=False)
load_weight = torch.load(model_path)
if type(load_weight)==OrderedDict:
    try:
        backbone.load_state_dict(load_weight)
    except:
        new_state_dict = OrderedDict()
        for n, v in load_weight.items():
            name = n.replace("module.","") 
            new_state_dict[name] = v
        backbone.load_state_dict(new_state_dict)
else:
    try:
        backbone.load_state_dict(load_weight.module.state_dict())
    except:
        backbone.load_state_dict(load_weight.state_dict())
_=backbone.eval()

img,ori_img = read_img(img_path)
output = backbone(img)

p,idx = torch.topk(output,1)
p = np.array(p.detach())[0]
idx = np.array(idx.detach())[0]
pred_name = label_dict[idx[0]]

print(pred_name)

Asian


In [16]:
from face_test.model_zoo.model_common import load_tensorRT, load_onnx, load_openvino,load_torch

In [54]:
# onnx
model_path = "race-mbf-arcface-sgd_221209_nf.onnx"
net = load_onnx.Onnx_session(model_path,input_mean=0.0, input_std=1.0,onnx_device='cuda')

aimg = (ori_img/255. - 0.5)/0.5
output = net(aimg)[0]

idx = np.argsort(output)[0,::-1][0]
p = output[0][idx]
pred_name = label_dict[idx]

print(pred_name)

providers: ['CUDAExecutionProvider']
Asian


In [68]:
# trt
model_path = "race-mbf-arcface-sgd_221209_nf.v7.trt"
net = load_tensorRT.TrtModel(model_path)

output = net(ori_img)[0]
idx = np.argsort(output)[0,::-1][0]
p = output[0][idx]
pred_name = label_dict[idx]

print(pred_name)

Asian


In [64]:
# openvino
# model_path = ["race-mbf-arcface-sgd_221209_nf.xml",
#              "race-mbf-arcface-sgd_221209_nf.bin"]
model_path = ["race-mbf-arcface-sgd_221209_nf_quantINT8.xml",
             "race-mbf-arcface-sgd_221209_nf_quantINT8.bin"]
net = load_openvino.Openvino(model_path,device='CPU')

output = net(ori_img)[0]
idx = np.argsort(output)[0,::-1][0]
p = output[0][idx]
pred_name = label_dict[idx]

print(pred_name)

Asian
